In [ ]:
#Writing a Neural Network Class from Scratch

import numpy as np
        
class NeuralNetwork:
    def __init__(self, neuronsperlayer): #Sets Up Weights and Biases of Neural Network
        self.noflayers = len(neuronsperlayer)
        #Xavier Initialisation
        self.w = [np.random.normal(0.0, (2.0 / (neuronsperlayer[i] + neuronsperlayer[i+1])) ** 0.5, size=(neuronsperlayer[i+1], neuronsperlayer[i])) for i in range(self.noflayers-1)]
        self.b = [np.zeros((neuronsperlayer[i+1], 1)) for i in range(self.noflayers-1)]
    
    @staticmethod
    def ReLu(x):
        return np.maximum(0, x)

    @staticmethod
    def derivative_ReLu(x):
        return (x > 0).astype(x.dtype)
        
    def forward(self, input_layers): #Forward Pass through Neural Network                 
        self.a = [input_layers]
        self.z = []
        
        for W, b in zip(self.w, self.b):
            Z = W @ self.a[-1] + b        
            self.z.append(Z)
            a = self.ReLu(Z)  
            self.a.append(a)
            
        return self.a[-1]         
    
    @staticmethod
    def loss(output_layers, targets): #Loss Function
        N1, N2 = targets.shape
        return 0.5 * np.sum((targets - output_layers)**2) / (N1 * N2)
    
    def backward(self, targets): #Backpropagation of Neural Network --> Provides Gradient of Loss Function
        N1, N2 = targets.shape
        delta = (self.a[-1] - targets) * self.derivative_ReLu(self.z[-1])
        self.grad_b = [delta.sum(axis=1, keepdims=True) / (N1 * N2)]
        self.grad_w = [delta @ self.a[-2].T / (N1 * N2)]

        for i in range(self.noflayers-2, 0, -1):
            delta = (self.w[i].T @ delta) * self.derivative_ReLu(self.z[i-1])
            self.grad_b.append(delta.sum(axis=1, keepdims=True) / (N1 * N2))
            self.grad_w.append(delta @ self.a[i-1].T / (N1 * N2))

        self.grad_w.reverse()
        self.grad_b.reverse()
        
    def update(self, learning_rate): #Updating the Weights and Biases of Neural Network
        for i in range(self.noflayers-1):
            self.w[i] -= learning_rate * self.grad_w[i]
            self.b[i] -= learning_rate * self.grad_b[i]
            
    def learn(self, training_data, target, epochs, learning_rate, batch_size): #Stochastic Gradient Descent 
        N = training_data.shape[1]
        
        for epoch in range(1, epochs+1):
            permutation = np.random.permutation(N)
            training_data, target = training_data[:, permutation], target[:, permutation]

            for i in range(0, N, batch_size):
                input_batch = training_data[:, i:i+batch_size]
                target_batch = target[:, i:i+batch_size]
                output_layers = self.forward(input_batch)
                self.backward(target_batch)
                self.update(learning_rate)

            if epoch % 500 == 0 or epoch == 1:
                loss = self.loss(self.forward(training_data), target)
                print(f"Epoch {epoch:3d} - Loss: {loss:.4f}")


In [ ]:
file_path = "/Users/davidseager/Desktop/Programming/Ising/Data/Ising Systems/"
temps = np.load("/Users/davidseager/Desktop/Programming/Ising/Data/Temperatures.npy")
sizeoflattices = 50
alldata = []

for temp in temps:
    path = file_path + f"L{sizeoflattices}_T{temp:.3f}.npy"
    lattices = np.load(path)
    alldata.append(lattices)
        
alldata = np.array(alldata, dtype=np.float32).reshape(-1, 50*50).T
        
epochs = 5000
batch_size = 256
learning_rate = 1e-2

net = NeuralNetwork([sizeoflattices**2, 512, 64, 2, 64, 512, sizeoflattices**2])
net.learn(training_data=alldata, target=alldata, epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)
torch.save(net.state_dict(), "autoencoder_fromScratch.pth")
